In [1]:
#Import the ne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

In [2]:
#create the credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
heart_db = 'heartdisease'

#create the engine
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, heart_db))

heart_df = pd.read_sql_query('select * from heartdisease',con=engine)

#dispose of the engine
engine.dispose()

In [3]:
#pulling the code from the lesson again
# Define the features and the outcome
X = heart_df.iloc[:, :13]
y = heart_df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace='?', value=0)

# Binarize `y` so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

#standardize the new X set
scaler = StandardScaler()
X_trans = scaler.fit_transform(X)

In [14]:
def gmm_test(n):
    gmm = GaussianMixture(n_components=n, random_state=357)
    clusters = gmm.fit_predict(X_trans)
    pca = PCA(n_components=2).fit_transform(X_trans)
    print('\nThe ARI is: {}'.format(metrics.adjusted_rand_score(y, clusters)))
    print('The silhouette is: {}'.format(metrics.silhouette_score(X_trans, clusters)))

In [15]:
gmm_test(2)


The ARI is: 0.4207322145049338
The silhouette is: 0.16118591340148433


For the n_clusters = 2 from KMeans:<br>
ARI score: 0.43661540614807665<br>
Silhouette score: 0.17440650461256255<br><br>
Agg cluster (complete):<br>
ARI score: -0.0010061512759301477<br>
Silhouette score: 0.34836188921330785<br><br>
Agg cluster (average):<br>
ARI score: -0.0010061512759301477<br>
Silhouette score: 0.34836188921330785<br><br>
Agg cluster (ward):<br>
ARI score: 0.146129913123814<br>
Silhouette score: 0.1387197366557222

KMeans has a slightly higher ARI while Agg cluster has the highest silhouette score. Based on the metrics, I would use the KMeans.

In [18]:
def gmm_test(t):
    gmm = GaussianMixture(n_components=2, random_state=357, covariance_type=t)
    clusters = gmm.fit_predict(X_trans)
    pca = PCA(n_components=2).fit_transform(X_trans)
    print('The ARI is: {}'.format(metrics.adjusted_rand_score(y, clusters)))
    print('The silhouette is: {}'.format(metrics.silhouette_score(X_trans, clusters)))

In [19]:
gmm_test('full')

The ARI is: 0.4207322145049338
The silhouette is: 0.16118591340148433


In [20]:
gmm_test('tied')

The ARI is: 0.4558104186161976
The silhouette is: 0.1671559472293439


In [21]:
gmm_test('diag')

The ARI is: 0.37878842718089933
The silhouette is: 0.15836933745078682


In [ ]:
gmm_test('sp')